In [1]:
import gym
import ma_gym 
import numpy as np
import random
import pandas as pd
from ma_gym.envs.traffic_junction import TrafficJunction  
from collections import defaultdict
import nashpy as nash
from dqn import DQNAgent
import matplotlib.pyplot as plt
from ppo import PPOAgent
from main import simulation_10_agents, simulation_4_agents, simulation_10_to_4_agents

Matplotlib created a temporary cache directory at /var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/matplotlib-1ryblxbm because the default path (/Users/gabesmithline/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# Seeds for reproducibility
seeds = [0, 1]

# Initialize dictionaries to store results
results_10_agents = defaultdict(list)
results_4_agents = defaultdict(list)
results_10_to_4_agents = defaultdict(list)


In [3]:
for seed in seeds:
    results_10_agents[seed] = simulation_10_agents(seed, False)  # Collect results from 10 agents simulation
    results_4_agents[seed]= simulation_4_agents(seed, False)    # Collect results from 4 agents simulation
    results_10_to_4_agents[seed] = simulation_10_to_4_agents(seed, False)  # Collect results from 10 to 4 agents simulation


/opt/homebrew/lib/python3.11/site-packages/gym/logger.py:34: UserWarning: WARN: Overriding environment TrafficJunction4-v0
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/opt/homebrew/lib/python3.11/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Generation 1/10
Solving for Nash Equilibrium in Generation 1/10
Computing Nash Equilibrium for 100 matches
Episode 1/5


/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 2/10
Solving for Nash Equilibrium in Generation 2/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 3/10
Solving for Nash Equilibrium in Generation 3/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 4/10
Solving for Nash Equilibrium in Generation 4/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 5/10
Solving for Nash Equilibrium in Generation 5/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 6/10
Solving for Nash Equilibrium in Generation 6/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 7/10
Solving for Nash Equilibrium in Generation 7/10
Computing Nash Equilibrium for 100 matches
Episode 1/

/opt/homebrew/lib/python3.11/site-packages/gym/logger.py:34: UserWarning: WARN: Overriding environment TrafficJunction4-v0
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/opt/homebrew/lib/python3.11/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


...

Generation 1/10
Solving for Nash Equilibrium in Generation 1/10
Computing Nash Equilibrium for 100 matches
Episode 1/5


/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 2/10
Solving for Nash Equilibrium in Generation 2/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 3/10
Solving for Nash Equilibrium in Generation 3/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 4/10
Solving for Nash Equilibrium in Generation 4/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 5/10
Solving for Nash Equilibrium in Generation 5/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 6/10
Solving for Nash Equilibrium in Generation 6/10
Computing Nash Equilibrium for 100 matches
Episode 1/5
Episode 2/5
Episode 3/5
Episode 4/5
Episode 5/5
Generation 7/10
Solving for Nash Equilibrium in Generation 7/10
Computing Nash Equilibrium for 100 matches
Episode 1/

/opt/homebrew/lib/python3.11/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Episode 0 done
Episode 1 done
Episode 2 done
Episode 3 done
Episode 4 done
Episode 5 done
Episode 5 done
Total steps: 13
Episode 6 done
Episode 7 done
Episode 8 done
Episode 9 done
Episode 10 done
Episode 11 done
Episode 12 done
Episode 12 done
Total steps: 13
Episode 13 done
Episode 14 done
Episode 14 done
Total steps: 13
Episode 15 done
Episode 16 done
Episode 17 done
Episode 18 done
Episode 19 done
Episode 20 done
Episode 21 done
Episode 22 done
Episode 23 done
Episode 24 done
Episode 24 done
Total steps: 13
Episode 25 done
Episode 26 done
Episode 27 done
Episode 28 done
Episode 28 done
Total steps: 13
Episode 29 done
Episode 30 done
Episode 31 done
Episode 32 done
Episode 33 done
Episode 34 done
Episode 35 done
Episode 35 done
Total steps: 13
Episode 36 done
Episode 37 done
Episode 38 done
Episode 39 done
Episode 40 done
Episode 41 done
Episode 42 done
Episode 43 done
Episode 44 done
Episode 45 done
Episode 46 done
Episode 47 done
Episode 48 done
Episode 49 done
Episode 50 done
Epi

In [1]:
def compute_averages_training(results_dict):
    averaged_results = defaultdict(list)
    for result in results_dict.values():
        for i, value in enumerate(result):
            if isinstance(value, (int, float)):  # Ensure the value is scalar
                averaged_results[i].append(value)
            else:
                # If it's not scalar, compute the mean of the list
                averaged_results[i].append(np.mean(value))

    # Calculate the mean of each metric
    for key in averaged_results.keys():
        averaged_results[key] = np.mean(averaged_results[key])
    return averaged_results

# Compute averages
average_results_10_agents = compute_averages_training(results_10_agents)
average_results_4_agents = compute_averages_training(results_4_agents)
average_results_10_to_4_agents = compute_averages_training(results_10_to_4_agents)

def plot_results_training(average_results, title):
    plt.figure(figsize=(10, 5))
    for metric, value in average_results.items():
        plt.plot(value, label=f'{metric} (Avg: {value:.2f})')
    plt.title(title)
    plt.xlabel('Episodes')
    plt.ylabel('Metric Values')
    plt.legend()
    plt.show()

plot_results_training(average_results_10_agents, 'Training Average Results for 10 Agents')
plot_results_training(average_results_4_agents, 'Training Average Results for 4 Agents')
plot_results_training(average_results_10_to_4_agents, 'Training Average Results for Transfer from 10 to 4 Agents')


NameError: name 'results_10_agents' is not defined